In [1]:
import aaai20
import numpy as np
import pandas as pd
from aaai20.io import build_filesystem, filename_query, filename_dataset, filename_model
from joblib import dump, load

from mercs.utils.encoding import code_to_query, encode_attribute, get_att, query_to_code

In [2]:
import mercs

from mercs import Mercs

In [3]:
nb_atts = 10**4

a = np.zeros((10**4, nb_atts))

idxs = np.random.randint(nb_atts, size=4500)

In [4]:
t = a.take(1, axis=1)

t.size

10000

In [5]:
# Get queries

fs = build_filesystem()

dataset='nltcs'

In [6]:
fn_qry = filename_query(dataset, suffix="default")
q_codes = np.load(fn_qry)

In [7]:
fn_train = filename_dataset(dataset, step=2, suffix="train", extension="csv")
df_train = pd.read_csv(fn_train, header=None, index_col=None)
train = df_train.values

fn_test = filename_dataset(dataset, step=2, suffix="test", extension="csv")
df = pd.read_csv(fn_test, header=None, index_col=None)

In [8]:
clf = Mercs(max_depth=2, nb_iterations=1, n_jobs=8, selection_algorithm="random", nb_targets=2, fraction_missing=0, verbose=1)
clf.fit(train, verbose=1)


/cw/dtailocal/repos/mercs/src/mercs/algo/induction.py:99: UserWarning: 
        Training is being parallellized using Joblib. Number of jobs = 8
        
  warnings.warn(msg)
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    2.5s remaining:    7.6s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    3.0s finished


In [9]:
clf.m_score

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.75602224, 0.        , 0.        , 0.75602224, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.73872761, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.73872761, 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.80481779, 0.        , 0.        , 0.        , 0.        ,
        0.80481779],
       [0.        , 0.        , 0.        , 0.        , 0.69363805,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.69363805,
        0.        ],
       [0.81593576, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.81593576, 

In [10]:
len(clf.m_list)

8

In [11]:
q_idx = 16
q_code = q_codes[q_idx]

In [12]:
test = df.values
test = test.astype(float)
target_ids = get_att(q_code, kind="targ").tolist()

y_true = test[:, target_ids].copy()  # Extract ground truth
test[
    :, target_ids
] = np.nan  # Ensure the answers never enter the algorithm

In [13]:
%%prun
y_pred = clf.predict(test, q_code=q_code, prediction_algorithm='it', max_steps=8)

         7432 function calls (7169 primitive calls) in 0.021 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       76    0.003    0.000    0.003    0.000 {method 'reduce' of 'numpy.ufunc' objects}
  327/199    0.002    0.000    0.008    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
       62    0.001    0.000    0.001    0.000 arraysetops.py:297(_unique1d)
       32    0.001    0.000    0.001    0.000 {method 'take' of 'numpy.ndarray' objects}
       18    0.001    0.000    0.001    0.000 arraysetops.py:483(in1d)
        6    0.001    0.000    0.001    0.000 {method 'predict' of 'sklearn.tree._tree.Tree' objects}
        8    0.001    0.000    0.004    0.000 _base.py:368(transform)
      343    0.001    0.000    0.001    0.000 graph.py:663(nodes)
      298    0.001    0.000    0.001    0.000 {built-in method numpy.array}
       18    0.000    0.000    0.001    0.000 vector_prediction.py:227(

In [14]:
clf.model_data

{'ind_time': 2.9878607299997384,
 'prd_time': 0.005617675999019411,
 'dia_time': 0.0011513260014908155,
 'infalgo_time': 0.0018166509998991387,
 'dsk_time': 0.013697676000447245,
 'inf_time': 0.022286442999757128}

In [15]:
%%prun
y_pred = clf.predict(test, q_code=q_code, prediction_algorithm='rw', max_steps=8, nb_walks=20)
y_pred

AssertionError: 

In [ ]:
clf.model_data